<a href="https://colab.research.google.com/github/Vaibhavishinde07/summeranalyticsweek1/blob/main/capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import time
import shutil
from datetime import datetime, timedelta
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot, column, row
from bokeh.io import push_notebook
import threading
import json
from math import radians, cos, sin, asin, sqrt
import warnings
warnings.filterwarnings('ignore')

# Enable Bokeh in notebooks
output_notebook()

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on earth (in km)"""
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

def create_sample_data():
    """Create sample parking data for demonstration"""
    np.random.seed(42)

    # Generate sample data
    dates = pd.date_range('2024-01-01', '2024-01-07', freq='1H')
    lot_ids = range(1, 11)  # 10 parking lots

    data = []
    for date in dates:
        for lot_id in lot_ids:
            # Simulate realistic parking patterns
            hour = date.hour
            is_peak = (8 <= hour <= 10) or (17 <= hour <= 19)
            is_weekend = date.weekday() >= 5

            # Base capacity and occupancy
            capacity = np.random.randint(50, 200)
            base_occupancy = 0.3 + 0.4 * np.sin(2 * np.pi * hour / 24)

            if is_peak:
                base_occupancy *= 1.5
            if is_weekend:
                base_occupancy *= 0.8

            occupancy = min(int(capacity * base_occupancy + np.random.normal(0, 10)), capacity)
            occupancy = max(0, occupancy)

            # Queue length
            queue_length = max(0, int(np.random.exponential(2) if occupancy > capacity * 0.8 else 0))

            # Geographic features (simulated)
            lat = 40.7128 + np.random.normal(0, 0.01)  # Around NYC
            lon = -74.0060 + np.random.normal(0, 0.01)

            data.append({
                'SystemCodeNumber': lot_id,
                'LastUpdatedDate': date.strftime('%d/%m/%Y'),
                'LastUpdatedTime': date.strftime('%H:%M:%S'),
                'Capacity': capacity,
                'Occupancy': occupancy,
                'QueueLength': queue_length,
                'VehicleType': np.random.choice(['Car', 'Motorcycle', 'Truck']),
                'IsSpecialDay': np.random.choice([0, 1], p=[0.9, 0.1]),
                'TrafficConditionNear': np.random.choice(['low', 'medium', 'high']),
                'Latitude': lat,
                'Longitude': lon
            })

    return pd.DataFrame(data)

def load_and_prepare_enhanced_data():
    """Load and prepare data with enhanced features including geographic proximity"""

    # Try to load existing data, otherwise create sample data
    try:
        df = pd.read_csv('parking_data.csv')
        print(" Loaded existing parking_data.csv")
    except FileNotFoundError:
        print(" Creating sample parking data...")
        df = create_sample_data()
        df.to_csv('parking_data.csv', index=False)
        print(" Sample data created and saved as parking_data.csv")

    # Parse timestamps
    df['timestamp'] = pd.to_datetime(
        df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
        dayfirst=True
    )

    # Rename columns
    col_map = {
        'SystemCodeNumber': 'lot_id',
        'Capacity': 'capacity',
        'Occupancy': 'occupancy',
        'QueueLength': 'queue_length',
        'IsSpecialDay': 'special_day',
        'TrafficConditionNear': 'traffic_level'
    }
    df = df.rename(columns=col_map)

    # Enhanced features
    df['occ_rate'] = df['occupancy'] / df['capacity']
    df['utilization_pressure'] = df['occupancy'] + df['queue_length']
    df['congestion_factor'] = df['occ_rate'] * df['queue_length']

    # Time-based features
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['is_peak_hour'] = ((df['hour'] >= 8) & (df['hour'] <= 10)) | ((df['hour'] >= 17) & (df['hour'] <= 19))
    df['is_weekend'] = df['day_of_week'].isin([5, 6])

    # One-hot encode categorical variables
    df = pd.get_dummies(df, columns=['VehicleType', 'traffic_level', 'special_day'], drop_first=True)

    # Calculate geographic proximity matrix for competitive analysis
    if 'Latitude' in df.columns and 'Longitude' in df.columns:
        unique_lots = df[['lot_id', 'Latitude', 'Longitude']].drop_duplicates()
        proximity_matrix = {}

        for _, lot1 in unique_lots.iterrows():
            proximity_matrix[lot1['lot_id']] = {}
            for _, lot2 in unique_lots.iterrows():
                if lot1['lot_id'] != lot2['lot_id']:
                    distance = haversine_distance(
                        lot1['Latitude'], lot1['Longitude'],
                        lot2['Latitude'], lot2['Longitude']
                    )
                    proximity_matrix[lot1['lot_id']][lot2['lot_id']] = distance

        # Add competitor information
        df['competitors_within_1km'] = 0
        df['competitors_within_500m'] = 0
        df['nearest_competitor_distance'] = 0.0

        for lot_id in df['lot_id'].unique():
            if lot_id in proximity_matrix:
                competitors = proximity_matrix[lot_id]
                within_1km = sum(1 for d in competitors.values() if d <= 1.0)
                within_500m = sum(1 for d in competitors.values() if d <= 0.5)
                nearest_distance = min(competitors.values()) if competitors else 999.0

                df.loc[df['lot_id'] == lot_id, 'competitors_within_1km'] = within_1km
                df.loc[df['lot_id'] == lot_id, 'competitors_within_500m'] = within_500m
                df.loc[df['lot_id'] == lot_id, 'nearest_competitor_distance'] = nearest_distance
    else:
        # Add default values if no geographic data
        df['competitors_within_1km'] = 0
        df['competitors_within_500m'] = 0
        df['nearest_competitor_distance'] = 999.0

    # Sort by timestamp for proper streaming order
    df = df.sort_values('timestamp').reset_index(drop=True)

    # Add sequential batch numbers for streaming
    df['batch_id'] = df.index // 30  # 30 records per batch for more frequent updates

    # Add timestamp string for easier handling
    df['timestamp_str'] = df['timestamp'].astype(str)

    print(f" Enhanced data prepared: {len(df)} records across {df['lot_id'].nunique()} parking lots")
    print(f" Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    print(f" Geographic features: {'Available' if 'Latitude' in df.columns else 'Not Available'}")

    return df

In [ ]:
class EnhancedParkingPriceModels:
    """Enhanced dynamic pricing models including competitive pricing"""

    def __init__(self):
        self.base_price = 10.0
        self.min_price = 5.0
        self.max_price = 20.0
        self.price_memory = {}  # Store price history for smoothing
        self.competitor_prices = {}  # Store competitor price information

    def update_price_memory(self, lot_id, price):
        """Update price memory for smoothing"""
        if lot_id not in self.price_memory:
            self.price_memory[lot_id] = []
        self.price_memory[lot_id].append(price)
        # Keep only last 5 prices for smoothing
        if len(self.price_memory[lot_id]) > 5:
            self.price_memory[lot_id] = self.price_memory[lot_id][-5:]

    def get_smoothed_price(self, lot_id, new_price, smoothing_factor=0.3):
        """Apply exponential smoothing to price changes"""
        if lot_id not in self.price_memory or len(self.price_memory[lot_id]) == 0:
            return new_price

        last_price = self.price_memory[lot_id][-1]
        smoothed_price = smoothing_factor * new_price + (1 - smoothing_factor) * last_price
        return smoothed_price

    def model1_enhanced_linear(self, prev_price, occupancy_rate, queue_length=0,
                               is_peak_hour=False, alpha=2.0):
        """
        Enhanced Model 1: Improved Linear Model with multiple factors
        """
        # Base price adjustment
        occupancy_adjustment = alpha * occupancy_rate

        # Queue pressure adjustment
        queue_adjustment = 0.5 * queue_length if queue_length > 0 else 0

        # Peak hour multiplier
        peak_multiplier = 1.2 if is_peak_hour else 1.0

        # Calculate new price
        new_price = (prev_price + occupancy_adjustment + queue_adjustment) * peak_multiplier

        return max(self.min_price, min(new_price, self.max_price))

    def model2_advanced_demand(self, prev_price, features, params=None):
        """
        Enhanced Model 2: Advanced Demand-Based Pricing
        """
        if params is None:
            params = {
                'alpha': 3.0,    # Occupancy weight
                'beta': 1.5,     # Queue weight
                'gamma': 0.8,    # Traffic penalty
                'delta': 2.0,    # Special day bonus
                'epsilon': 1.2,  # Vehicle type weight
                'zeta': 1.5,     # Peak hour multiplier
                'eta': 0.7       # Weekend adjustment
            }

        # Extract features with defaults
        occ_rate = features.get('occ_rate', 0)
        queue_length = features.get('queue_length', 0)
        congestion_factor = features.get('congestion_factor', 0)
        traffic_high = features.get('traffic_level_high', 0)
        special_day = features.get('special_day_1', 0)
        is_peak_hour = features.get('is_peak_hour', False)
        is_weekend = features.get('is_weekend', False)

        # Vehicle type aggregation
        vehicle_weights = sum(
            features.get(key, 0) for key in features.keys()
            if key.startswith('VehicleType_')
        )

        # Advanced demand calculation
        base_demand = (
            params['alpha'] * occ_rate +
            params['beta'] * min(queue_length / 10.0, 1.0) +  # Normalize queue
            params['gamma'] * congestion_factor -
            params['delta'] * traffic_high +
            params['epsilon'] * special_day +
            params['zeta'] * vehicle_weights
        )

        # Time-based adjustments
        time_multiplier = 1.0
        if is_peak_hour:
            time_multiplier *= params['zeta']
        if is_weekend:
            time_multiplier *= params['eta']

        # Apply sophisticated normalization
        normalized_demand = np.tanh(base_demand / 8.0)  # Smooth sigmoid

        # Calculate price with multiple factors
        price = self.base_price * (1 + 0.6 * normalized_demand) * time_multiplier

        return np.clip(price, self.min_price, self.max_price)

    def model3_competitive_pricing(self, lot_id, prev_price, features, competitor_data=None):
        """
        Model 3: Competitive Pricing with Geographic Intelligence
        """
        # Start with Model 2 as base
        base_price = self.model2_advanced_demand(prev_price, features)

        # Extract competitive features
        occ_rate = features.get('occ_rate', 0)
        competitors_within_1km = features.get('competitors_within_1km', 0)
        competitors_within_500m = features.get('competitors_within_500m', 0)
        nearest_distance = features.get('nearest_competitor_distance', 999)

        # Competitive adjustments
        competitive_adjustment = 1.0

        # If lot is nearly full, check competitor pricing
        if occ_rate > 0.8:
            # High occupancy - can afford to increase price
            if competitors_within_500m > 0:
                competitive_adjustment *= 1.15  # Increase by 15%
            elif competitors_within_1km > 0:
                competitive_adjustment *= 1.10  # Increase by 10%

        # If lot has low occupancy, be more competitive
        elif occ_rate < 0.3:
            if competitors_within_500m > 2:
                competitive_adjustment *= 0.90  # Decrease by 10%
            elif competitors_within_1km > 1:
                competitive_adjustment *= 0.95  # Decrease by 5%

        # Distance-based adjustment
        if nearest_distance < 0.2:  # Very close competitors
            if occ_rate > 0.6:
                competitive_adjustment *= 1.05
            else:
                competitive_adjustment *= 0.95

        # Apply competitive adjustment
        competitive_price = base_price * competitive_adjustment

        # Rerouting suggestion logic
        reroute_suggestion = None
        if occ_rate > 0.9 and competitors_within_1km > 0:
            reroute_suggestion = f"Consider nearby lots within 1km (Distance: {nearest_distance:.2f}km)"

        return {
            'price': np.clip(competitive_price, self.min_price, self.max_price),
            'reroute_suggestion': reroute_suggestion,
            'competitive_factor': competitive_adjustment
        }

In [ ]:
class EnhancedStreamingPipeline:
    """Enhanced real-time pricing pipeline"""

    def __init__(self, pricing_models):
        self.pricing_models = pricing_models
        self.lot_prices = {}
        self.results = []
        self.price_history = {}

    def initialize_lot_prices(self, lot_ids):
        """Initialize prices and history for all lots"""
        for lot_id in lot_ids:
            self.lot_prices[lot_id] = self.pricing_models.base_price
            self.price_history[lot_id] = [self.pricing_models.base_price]

    def process_batch_advanced(self, batch_data, model_choice='model3'):
        """Process a batch of data with advanced pricing logic"""

        batch_results = []

        for _, row in batch_data.iterrows():
            lot_id = row['lot_id']
            prev_price = self.lot_prices.get(lot_id, self.pricing_models.base_price)

            # Prepare features
            features = row.to_dict()

            # Calculate price based on model choice
            if model_choice == 'model1':
                new_price = self.pricing_models.model1_enhanced_linear(
                    prev_price,
                    row['occ_rate'],
                    row.get('queue_length', 0),
                    row.get('is_peak_hour', False)
                )
                result = {
                    'price': new_price,
                    'reroute_suggestion': None,
                    'competitive_factor': 1.0
                }
            elif model_choice == 'model2':
                new_price = self.pricing_models.model2_advanced_demand(prev_price, features)
                result = {
                    'price': new_price,
                    'reroute_suggestion': None,
                    'competitive_factor': 1.0
                }
            else:  # model3
                result = self.pricing_models.model3_competitive_pricing(
                    lot_id, prev_price, features
                )

            # Apply price smoothing
            smoothed_price = self.pricing_models.get_smoothed_price(
                lot_id, result['price']
            )

            # Update price memory
            self.pricing_models.update_price_memory(lot_id, smoothed_price)
            self.lot_prices[lot_id] = smoothed_price

            # Store result
            batch_result = {
                'lot_id': lot_id,
                'timestamp_str': row['timestamp_str'],
                'occupancy': row['occupancy'],
                'capacity': row['capacity'],
                'occ_rate': row['occ_rate'],
                'queue_length': row.get('queue_length', 0),
                'price': smoothed_price,
                'raw_price': result['price'],
                'reroute_suggestion': result.get('reroute_suggestion'),
                'competitive_factor': result.get('competitive_factor', 1.0),
                'is_peak_hour': row.get('is_peak_hour', False),
                'competitors_nearby': row.get('competitors_within_1km', 0)
            }

            batch_results.append(batch_result)
            self.results.append(batch_result)

        return batch_results

    def run_enhanced_streaming(self, df, model_choice='model3'):
        """Run enhanced streaming simulation"""

        print(f" Starting enhanced streaming with {model_choice.upper()}")

        # Initialize prices
        unique_lots = df['lot_id'].unique()
        self.initialize_lot_prices(unique_lots)

        # Process batches
        total_batches = df['batch_id'].nunique()

        for i, batch_id in enumerate(sorted(df['batch_id'].unique())):
            batch_data = df[df['batch_id'] == batch_id]

            # Process batch
            batch_results = self.process_batch_advanced(batch_data, model_choice)

            # Progress indication
            if i % 10 == 0:
                avg_price = np.mean([r['price'] for r in batch_results])
                print(f"Batch {i+1}/{total_batches} - Avg Price: ${avg_price:.2f}")

            # Simulate real-time delay
            time.sleep(0.01)  # Reduced for faster demo

        print(f" Enhanced streaming complete: {len(self.results)} records processed")
        return self.results

In [ ]:
def create_enhanced_visualizations(results, model_name="Enhanced Model"):
    """Create comprehensive visualizations"""

    df_results = pd.DataFrame(results)
    if df_results.empty:
        print(" No results to visualize")
        return

    df_results['timestamp'] = pd.to_datetime(df_results['timestamp_str'])

    print(f" Creating enhanced visualizations for {model_name}")


    def plot_competitive_analysis():
        top_lots = df_results['lot_id'].value_counts().head(6).index

        plots = []
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

        for i, lot_id in enumerate(top_lots):
            lot_data = df_results[df_results['lot_id'] == lot_id].sort_values('timestamp')

            if len(lot_data) == 0:
                continue

            p = figure(
                x_axis_type='datetime',
                title=f'Lot {lot_id} - Competitive Pricing Analysis',
                width=450,
                height=300,
                tools="pan,wheel_zoom,box_zoom,reset,save"
            )

            # Price line
            p.line(lot_data['timestamp'], lot_data['price'],
                   line_width=2, color=colors[i % len(colors)],
                   legend_label='Final Price')

            # Raw price (before smoothing)
            p.line(lot_data['timestamp'], lot_data['raw_price'],
                   line_width=1, color=colors[i % len(colors)],
                   alpha=0.5, line_dash='dashed',
                   legend_label='Raw Price')

            # Competitive factor indicator
            competitive_lots = lot_data[lot_data['competitive_factor'] != 1.0]
            if not competitive_lots.empty:
                p.circle(competitive_lots['timestamp'], competitive_lots['price'],
                        size=8, color='red', alpha=0.7,
                        legend_label='Competitive Adjustment')

            # Peak hour indicators
            peak_lots = lot_data[lot_data['is_peak_hour'] == True]
            if not peak_lots.empty:
                p.triangle(peak_lots['timestamp'], peak_lots['price'],
                          size=6, color='orange', alpha=0.6,
                          legend_label='Peak Hour')

            p.legend.location = "top_left"
            p.legend.click_policy = "hide"
            p.xaxis.axis_label = "Time"
            p.yaxis.axis_label = "Price ($)"

            plots.append(p)

        if plots:
            grid = gridplot([plots[i:i+2] for i in range(0, len(plots), 2)])
            show(grid)

    def plot_price_efficiency():
        p = figure(
            title='Price Efficiency: Occupancy vs Price Relationship',
            width=600,
            height=400,
            tools="pan,wheel_zoom,box_zoom,reset,save"
        )

        # Color by competitive factor
        colors = ['red' if cf > 1.05 else 'blue' if cf < 0.95 else 'green'
                  for cf in df_results['competitive_factor']]

        p.circle(df_results['occ_rate'], df_results['price'],
                size=8, color=colors, alpha=0.6)

        # Add trend line
        if len(df_results) > 1:
            z = np.polyfit(df_results['occ_rate'], df_results['price'], 2)  # Quadratic fit
            x_trend = np.linspace(df_results['occ_rate'].min(), df_results['occ_rate'].max(), 100)
            p.line(x_trend, np.poly1d(z)(x_trend),
                   line_width=2, color='black', alpha=0.8)

        p.xaxis.axis_label = "Occupancy Rate"
        p.yaxis.axis_label = "Price ($)"

        show(p)

    def plot_revenue_analysis():
        df_results['potential_revenue'] = df_results['price'] * df_results['occupancy']

        p = figure(
            title='Revenue Optimization Analysis',
            width=800,
            height=400,
            tools="pan,wheel_zoom,box_zoom,reset,save"
        )

        # Revenue by lot
        lot_revenue = df_results.groupby('lot_id')['potential_revenue'].sum().reset_index()
        lot_revenue = lot_revenue.sort_values('potential_revenue', ascending=False).head(10)

        p.vbar(x=lot_revenue['lot_id'].astype(str), top=lot_revenue['potential_revenue'],
               width=0.8, color='navy', alpha=0.7)

        p.xaxis.axis_label = "Lot ID"
        p.yaxis.axis_label = "Total Potential Revenue ($)"
        p.xaxis.major_label_orientation = 45

        show(p)

    # Generate all visualizations
    try:
        plot_competitive_analysis()
        plot_price_efficiency()
        plot_revenue_analysis()
    except Exception as e:
        print(f" Visualization error: {e}")
        print("Continuing with text-based analysis...")

In [ ]:
def run_complete_model_comparison(df):
    """Run all three models and compare results"""

    print(" Running Complete Model Comparison")


    # Initialize enhanced models
    enhanced_models = EnhancedParkingPriceModels()

    # Run all models
    results = {}

    for model_name in ['model1', 'model2', 'model3']:
        print(f"\n Running {model_name.upper()}...")

        pipeline = EnhancedStreamingPipeline(enhanced_models)
        model_results = pipeline.run_enhanced_streaming(df, model_name)
        results[model_name] = model_results

    # Compare results
    print("\n MODEL COMPARISON RESULTS")


    for model_name, model_results in results.items():
        df_model = pd.DataFrame(model_results)

        print(f"\n{model_name.upper()} Results:")
        print(f"  Records Processed: {len(df_model)}")
        print(f"  Average Price: ${df_model['price'].mean():.2f}")
        print(f"  Price Range: ${df_model['price'].min():.2f} - ${df_model['price'].max():.2f}")
        print(f"  Price Volatility: {df_model['price'].std():.2f}")

        if 'competitive_factor' in df_model.columns:
            competitive_adjustments = df_model[df_model['competitive_factor'] != 1.0]
            print(f"  Competitive Adjustments: {len(competitive_adjustments)} ({len(competitive_adjustments)/len(df_model)*100:.1f}%)")

        if 'reroute_suggestion' in df_model.columns:
            reroute_suggestions = df_model[df_model['reroute_suggestion'].notna()]
            print(f"  Reroute Suggestions: {len(reroute_suggestions)}")

        # Calculate efficiency metrics
        total_revenue = (df_model['price'] * df_model['occupancy']).sum()
        avg_utilization = df_model['occ_rate'].mean()

        print(f"  Total Potential Revenue: ${total_revenue:.2f}")
        print(f"  Average Utilization: {avg_utilization:.2%}")
        print(f"  Revenue Efficiency: ${total_revenue/len(df_model):.2f}/record")

    # Create comparison visualizations
    print("\n Generating comparison visualizations...")

    try:
        # Combined visualization
        fig = figure(
            title='Model Comparison: Price Distribution',
            width=800,
            height=400,
            tools="pan,wheel_zoom,box_zoom,reset,save"
        )

        colors = ['blue', 'red', 'green']
        for i, (model_name, model_results) in enumerate(results.items()):
            df_model = pd.DataFrame(model_results)
            hist, edges = np.histogram(df_model['price'], bins=30, density=True)
            fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
                    fill_color=colors[i], alpha=0.5, legend_label=model_name.upper())

        fig.legend.location = "top_right"
        fig.xaxis.axis_label = "Price ($)"
        fig.yaxis.axis_label = "Density"

        show(fig)
    except Exception as e:
        print(f" Comparison visualization error: {e}")

    return results

In [ ]:
def main():
    """Main execution function"""

    print(" Enhanced Dynamic Parking Pricing System")


    # Load enhanced data
    df_enhanced = load_and_prepare_enhanced_data()

    # Run complete model comparison
    all_results = run_complete_model_comparison(df_enhanced)

    # Create enhanced visualizations for Model 3
    if 'model3' in all_results:
        create_enhanced_visualizations(all_results['model3'], "Model 3 - Competitive")

    print("\n Enhanced Dynamic Parking Pricing System Complete!")
    print(" All models implemented with competitive intelligence!")

    return all_results

# Run the enhanced system
if __name__ == "__main__":
    enhanced_results = main()

 Enhanced Dynamic Parking Pricing System
 Creating sample parking data...
 Sample data created and saved as parking_data.csv
 Enhanced data prepared: 1450 records across 10 parking lots
 Time range: 2024-01-01 00:00:00 to 2024-01-07 00:00:00
 Geographic features: Available
 Running Complete Model Comparison

 Running MODEL1...
 Starting enhanced streaming with MODEL1
Batch 1/49 - Avg Price: $10.89
Batch 11/49 - Avg Price: $19.97
Batch 21/49 - Avg Price: $20.00
Batch 31/49 - Avg Price: $20.00
Batch 41/49 - Avg Price: $20.00
 Enhanced streaming complete: 1450 records processed

 Running MODEL2...
 Starting enhanced streaming with MODEL2
Batch 1/49 - Avg Price: $15.93
Batch 11/49 - Avg Price: $12.94
Batch 21/49 - Avg Price: $13.34
Batch 31/49 - Avg Price: $14.13
Batch 41/49 - Avg Price: $9.99
 Enhanced streaming complete: 1450 records processed

 Running MODEL3...
 Starting enhanced streaming with MODEL3
Batch 1/49 - Avg Price: $9.82
Batch 11/49 - Avg Price: $13.05
Batch 21/49 - Avg Price

 Creating enhanced visualizations for Model 3 - Competitive



 Enhanced Dynamic Parking Pricing System Complete!
 All models implemented with competitive intelligence!
